In [1]:
# ⬇️ 필요한 라이브러리 설치 (최초 1회만)
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# ⬇️ Gmail 자동화 통합 코드
import os
import base64
from google.colab import files
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# === Step 0. credentials.json 업로드 ===
print("⬆️ credentials.json 파일을 업로드 해주세요.")
uploaded = files.upload()

# === Step 1. Gmail 인증 ===
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
def gmail_authenticate():
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    service = build('gmail', 'v1', credentials=creds)
    return service

service = gmail_authenticate()
print("✅ Gmail 인증 완료")

# === Step 2. 메일 검색 및 첨부파일 다운로드 ===
def search_emails(service, query, max_results=5):
    results = service.users().messages().list(userId='me', q=query, maxResults=max_results).execute()
    messages = results.get('messages', [])
    return messages

def get_attachments(service, msg_id, save_dir="downloads"):
    os.makedirs(save_dir, exist_ok=True)
    msg = service.users().messages().get(userId='me', id=msg_id).execute()
    payload = msg.get('payload', {})
    parts = payload.get('parts', [])

    for part in parts:
        if part.get("filename") and 'attachmentId' in part.get("body", {}):
            filename = part["filename"]
            if not filename.endswith(".xlsx"):
                continue
            attach_id = part["body"]["attachmentId"]
            attachment = service.users().messages().attachments().get(
                userId='me', messageId=msg_id, id=attach_id).execute()
            data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
            path = os.path.join(save_dir, filename)
            with open(path, 'wb') as f:
                f.write(data)
            print(f"📥 첨부파일 저장 완료: {path}")
            return path
    return None

# === Step 3. 메일 작성 및 자동 발송 ===
def create_message_with_attachment(to, subject, body_text, file_path):
    message = MIMEMultipart()
    message['to'] = to
    message['subject'] = subject
    message.attach(MIMEText(body_text, 'plain'))

    with open(file_path, 'rb') as f:
        part = MIMEApplication(f.read(), Name=os.path.basename(file_path))
    part['Content-Disposition'] = f'attachment; filename="{os.path.basename(file_path)}"'
    message.attach(part)

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw}

# === Step 4. 전체 실행 ===
query = 'subject:발주서 has:attachment is:unread'
receiver_email = 'jis9984@hanjin.com'
subject = '발주서 전달드립니다'
body = '안녕하세요, 첨부된 발주서를 전달드립니다.'

messages = search_emails(service, query)
if messages:
    filepath = get_attachments(service, messages[0]['id'])
    if filepath:
        msg = create_message_with_attachment(receiver_email, subject, body, filepath)
        send = service.users().messages().send(userId='me', body=msg).execute()
        print(f"📤 메일 발송 완료! 메시지 ID: {send['id']}")
else:
    print("📭 '발주서' 제목을 가진 미확인 메일이 없습니다.")


⬆️ credentials.json 파일을 업로드 해주세요.


Saving client_secret_1094072175836-mi38aq3tu5spmqku1cuuq1d57pq6os01.apps.googleusercontent.com.json to client_secret_1094072175836-mi38aq3tu5spmqku1cuuq1d57pq6os01.apps.googleusercontent.com.json


FileNotFoundError: [Errno 2] No such file or directory: 'credentials.json'

In [5]:
# # ✅ STEP 1: 라이브러리 설치
# !pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

# ✅ STEP 2: 필수 모듈 임포트
import os
import base64
import json
from google.colab import auth, files
from googleapiclient.discovery import build
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication
from google.oauth2.credentials import Credentials

# ✅ STEP 3: Google Colab 인증 (Google 계정 로그인)
print("🔐 Google 계정으로 로그인하세요.")
auth.authenticate_user()

# ✅ STEP 4: credentials.json 업로드 및 Gmail API 인증 설정
print("⬆️ credentials.json 또는 client_secret_*.json 파일 업로드")
uploaded = files.upload()

# credentials.json 파일명 자동 변경
for filename in uploaded.keys():
    if filename.startswith("client_secret") or filename == "credentials.json":
        os.rename(filename, "credentials.json")
        print(f"✅ '{filename}' → 'credentials.json' 으로 이름 변경 완료")

# credentials.json에서 인증 정보 추출하여 Gmail API 연결
with open("credentials.json", "r") as f:
    creds_info = json.load(f)

creds = Credentials.from_authorized_user_info(info=creds_info, scopes=["https://www.googleapis.com/auth/gmail.modify"])
service = build('gmail', 'v1', credentials=creds)
print("✅ Gmail API 인증 성공!")

# ✅ STEP 5: 메일 검색 + 첨부파일 다운로드 함수 정의
def search_emails(service, query, max_results=5):
    results = service.users().messages().list(userId='me', q=query, maxResults=max_results).execute()
    return results.get('messages', [])

def get_attachments(service, msg_id, save_dir="downloads"):
    os.makedirs(save_dir, exist_ok=True)
    msg = service.users().messages().get(userId='me', id=msg_id).execute()
    parts = msg.get('payload', {}).get('parts', [])

    for part in parts:
        filename = part.get("filename")
        if filename and filename.endswith(".xlsx") and 'attachmentId' in part.get("body", {}):
            attach_id = part["body"]["attachmentId"]
            attachment = service.users().messages().attachments().get(
                userId='me', messageId=msg_id, id=attach_id).execute()
            data = base64.urlsafe_b64decode(attachment['data'].encode('UTF-8'))
            path = os.path.join(save_dir, filename)
            with open(path, 'wb') as f:
                f.write(data)
            print(f"📥 첨부파일 저장 완료: {path}")
            return path
    print("⚠️ 첨부파일(.xlsx) 없음")
    return None

# ✅ STEP 6: 자동 메일 생성 및 전송
def create_message_with_attachment(to, subject, body_text, file_path):
    message = MIMEMultipart()
    message['to'] = to
    message['subject'] = subject
    message.attach(MIMEText(body_text, 'plain'))

    with open(file_path, 'rb') as f:
        part = MIMEApplication(f.read(), Name=os.path.basename(file_path))
    part['Content-Disposition'] = f'attachment; filename="{os.path.basename(file_path)}"'
    message.attach(part)

    raw = base64.urlsafe_b64encode(message.as_bytes()).decode()
    return {'raw': raw}

# ✅ STEP 7: 실행 조건 및 발송 트리거
query = 'subject:잔업예정명단 has:attachment is:unread'
receiver_email = 'partner@example.com'  # 📬 자동 발송 대상자 이메일
subject = '잔업예정명단Test 전달드립니다'
body = '안녕하세요, 첨부된 발주서를 전달드립니다.'

messages = search_emails(service, query)
if messages:
    filepath = get_attachments(service, messages[0]['id'])
    if filepath:
        msg = create_message_with_attachment(receiver_email, subject, body, filepath)
        send = service.users().messages().send(userId='me', body=msg).execute()
        print(f"📤 메일 발송 완료! 메시지 ID: {send['id']}")
else:
    print("📭 '발주서' 제목의 미확인 메일이 없습니다.")


🔐 Google 계정으로 로그인하세요.
⬆️ credentials.json 또는 client_secret_*.json 파일 업로드


Saving client_secret_1094072175836-mi38aq3tu5spmqku1cuuq1d57pq6os01.apps.googleusercontent.com.json to client_secret_1094072175836-mi38aq3tu5spmqku1cuuq1d57pq6os01.apps.googleusercontent.com (1).json
✅ 'client_secret_1094072175836-mi38aq3tu5spmqku1cuuq1d57pq6os01.apps.googleusercontent.com (1).json' → 'credentials.json' 으로 이름 변경 완료


ValueError: Authorized user info was not in the expected format, missing fields refresh_token, client_secret, client_id.